### Домашнее задание

#### 1. Для чего и в каких случаях полезны различные варианты усреднения для метрик качества классификации: micro, macro, weighted?

В случае многоклассовой классификации для получения общей F1. Для каждого класса на основе их собственных precision и recall расчитываются отдельные F1 и на основе их расчитывается общий для всех классов F1. При этом micro-F1 из-за стирания различий между FP и FN мне кажется неприменима на небинарных классах. Она еквивалентна accuracy и на несбалансированных выборках так же смысла не имеет. macro-F1 есть среднее арифметическое F1 каждого класса, весов не учитывает, так же имеет смысл только на сбалансированных выборках. weighted-F1 придаёт каждому классу вес в зависимости от количества полученных предиктором значений этого класса что будет полезно для несбалансированных выборок.

#### 2. В чём разница между моделями xgboost, lightgbm и catboost или какие их основные особенности?

* CatBoost реализует алгоритм упорядоченного бустинга, в следсвие чего не переобучается на малом количестве данных как другие алгоритмы. Обрабатывается категориальные признаки при помощи one-hot кодирования, при этом он принимает строковые значения признаков. Нечувствительность к пропускам.
* LightGBM так же может обрабатывать категориальные признаки, но не допускает строковых значений и не применяет one-hot кодирования, при этом работет быстрее CatBoost.
* XGBoost сам не может обрабатывать категориальные признаки, он принимает только числовые значения, следовательно, данные для него необходимо готовить отдельно.

### Задания по курсовому проекту (*)

#### a) Построить baseline model

Разбить данные из train на обучающую и контрольную выборки.
Выбрать алгоритм с параметрами по умолчанию.
Обучить его на обучающей выборке с минимальной предобработкой, необходимой для выбранного алгоритма.
Сделать предсказания для контрольной выборки и рассчитать метрики precision, recall и f1-score.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import catboost as catboost
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from sklearn.feature_selection import RFE
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_classification_report(y_train_true, y_train_pred, y_test_true, y_test_pred):
    print('TRAIN\n\n' + classification_report(y_train_true, y_train_pred))
    print('TEST\n\n' + classification_report(y_test_true, y_test_pred))
    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_test_true, y_test_pred))

In [3]:
train_df = pd.read_csv('H:\\GeekBrains\\Основное обучение\\II четверть\\Библиотеки Python для Data Science продолжение\\Урок 1. Введение в задачу классификации. Постановка задачи и подготовка данных\course_project\\course_project_train.csv')
train_df

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,Own Home,482087.0,NaN,0.0,11.0,26.3,685960.0,1.0,NaN,1.0,debt consolidation,Short Term,99999999.0,47386.0,7914.0,749.0,0
1,Own Home,1025487.0,10+ years,0.0,15.0,15.3,1181730.0,0.0,NaN,0.0,debt consolidation,Long Term,264968.0,394972.0,18373.0,737.0,1
2,Home Mortgage,751412.0,8 years,0.0,11.0,35.0,1182434.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,308389.0,13651.0,742.0,0
3,Own Home,805068.0,6 years,0.0,8.0,22.5,147400.0,1.0,NaN,1.0,debt consolidation,Short Term,121396.0,95855.0,11338.0,694.0,0
4,Rent,776264.0,8 years,0.0,13.0,13.6,385836.0,1.0,NaN,0.0,debt consolidation,Short Term,125840.0,93309.0,7180.0,719.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,Rent,402192.0,< 1 year,0.0,3.0,8.5,107866.0,0.0,NaN,0.0,other,Short Term,129360.0,73492.0,1900.0,697.0,0
7496,Home Mortgage,1533984.0,1 year,0.0,10.0,26.5,686312.0,0.0,43.0,0.0,debt consolidation,Long Term,444048.0,456399.0,12783.0,7410.0,1
7497,Rent,1878910.0,6 years,0.0,12.0,32.1,1778920.0,0.0,NaN,0.0,buy a car,Short Term,99999999.0,477812.0,12479.0,748.0,0
7498,Home Mortgage,NaN,NaN,0.0,21.0,26.5,1141250.0,0.0,NaN,0.0,debt consolidation,Short Term,615274.0,476064.0,37118.0,NaN,0


In [4]:
target = 'Credit Default'
X = train_df.drop(columns=target)
y = train_df[target]
X.loc[X['Years in current job'].isna(), 'Years in current job'] = '10+ years'#преобладает
train_df = train_df.drop(['Purpose'], axis=1)#половина значений числа, половина строки, непонятно что с этим делать

In [5]:
%%time
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, shuffle=True, random_state=22)
catb = catboost.CatBoostClassifier(silent=True, random_state=22)
catb.fit(X_train, y_train, cat_features=[0, 2, 10, 11])
y_train_pred = catb.predict(X_train)
y_valid_pred = catb.predict(X_valid)

Wall time: 54.9 s


In [6]:
print('precision, recall и f1-score для контрольной выборки\n\n' + classification_report(y_valid, y_valid_pred))

precision, recall и f1-score для контрольной выборки

              precision    recall  f1-score   support

           0       0.78      0.97      0.86      1767
           1       0.80      0.31      0.45       708

    accuracy                           0.78      2475
   macro avg       0.79      0.64      0.66      2475
weighted avg       0.78      0.78      0.74      2475



#### b) Создать новые признаки (feature generation).**

1. Признаки на основе финансовых коэффициентов для оценки кредитоспособности клиента:
    * Чистый долг = Размер кредита - Размер дохода
    * Период выплаты кредита из дохода = Размер кредита / Годовой доход
    * Доля платежа по кредиту в доходе = Годовой платеж по кредиту / Годовой доход
2. Новые признаки на основе числовых признаков:
    * используя условия на значения признака
    * квантование (binning) - cut, qcut
    * трансформация (log, pow, масштабирование)
3. Новые признаки на основе категориальных признаков:
    * Label Encoding (категориям присваиваются целые числа)
    * One-hot Encoding (дамми-кодирование)
    * Frequency Encoding (категория заменяется на частоту наблюдений в этой категории)
    * Likelihood Encoding (категория заменяется на среднее значение целевого признака/долю целевого признака)
4. Новые признаки на основе взаимодействия признаков:
    * используя деление, разность, сумму, произведение разных числовых признаков
    * категория заменяется на статистику другого признака в этой категории (min, max, mean, std)
    * используя конкатенацию категории разных признаков
    * используя методы кластеризации (PCA)

In [7]:
#1. Признаки на основе финансовых коэффициентов для оценки кредитоспособности клиента
#Чистый долг = Размер кредита - Размер дохода - непонятрно, почему кредит минус годовой доход это чистый долг?

#Период выплаты кредита из дохода = сумма кредита / годовой доход
train_df['loan_repayment_period'] = train_df['Current Loan Amount'] / train_df['Annual Income'] 

#Доля платежа по кредиту в доходе = Годовой платеж по кредиту / Годовой доход
train_df['loan_part'] = (train_df['Monthly Debt']*12) / train_df['Annual Income']

In [8]:
#2. Новые признаки на основе числовых признаков

#доход до 200000 в год - беднота
train_df.loc[train_df['Annual Income'] < 200000, 'social_class'] = 0 
#доход от 200000 до 1500000 в год - зажиточные граждане
train_df.loc[(train_df['Annual Income'] >= 200000) & (train_df['Annual Income'] < 1500000), 'social_class'] = 1
#доход более 1500000 в год - зажиточные граждане
train_df.loc[train_df['Annual Income'] >= 1500000, 'social_class'] = 2

#размер кредита по отношению к годовому доходу (нагрузка на бюджет)
train_df['budget_burdening'] = train_df['Annual Income'] / train_df['Current Loan Amount']

#проходил процедуру банкротства
train_df['had_bankruptcies'] = 0
train_df.loc[train_df['Bankruptcies'] != 0, 'had_bankruptcies'] = 1

In [9]:
#3. Новые признаки на основе категориальных признаков 
#label encoding
#степень обременения home_ownership_cat: 0 - есть иппотека, 1 - съём жилья, 2 - собственник жилья
train_df.loc[(train_df['Home Ownership'] == 'Home Mortgage') | (train_df['Home Ownership'] == 'Have Mortgage'), 'home_ownership_cat'] = 0
train_df.loc[train_df['Home Ownership'] == 'Rent', 'home_ownership_cat'] = 1
train_df.loc[train_df['Home Ownership'] == 'Own Home', 'home_ownership_cat'] = 2
train_df = train_df.drop(['Home Ownership'], axis=1)

#Likelihood encoding
train_df.loc[train_df['Years in current job'] == '10'] = 10
train_df.loc[train_df['Years in current job'] == '2 years'] = 2
train_df.loc[train_df['Years in current job'] == '3 years'] = 3
train_df.loc[train_df['Years in current job'] == '< 1 year'] = 0.5
train_df.loc[train_df['Years in current job'] == '5 years'] = 5
train_df.loc[train_df['Years in current job'] == '1 year'] = 1
train_df.loc[train_df['Years in current job'] == '4 years'] = 4
train_df.loc[train_df['Years in current job'] == '6 years'] = 6
train_df.loc[train_df['Years in current job'] == '7 years'] = 7
train_df.loc[train_df['Years in current job'] == '8 years'] = 8
train_df.loc[train_df['Years in current job'] == '9 years'] = 9
train_df.loc[train_df['Years in current job'] == '10+ years'] = 11

train_df.loc[train_df['Term'] == 'Short Term', 'Term'] = 0.5
train_df.loc[train_df['Term'] == 'Long Term', 'Term'] = 11

#доля дефолтов по отношению к home_ownership_cat
total_defaults = len(train_df.loc[train_df[target] == 1]) #всего дефолтов
home_ownership_cat_0 = len(train_df.loc[(train_df['home_ownership_cat'] == 0) & train_df[target] == 1])#дефолтов по категории 0
home_ownership_cat_1 = len(train_df.loc[(train_df['home_ownership_cat'] == 1) & train_df[target] == 1])#дефолтов по категории 1
home_ownership_cat_2 = len(train_df.loc[(train_df['home_ownership_cat'] == 2) & train_df[target] == 1])#дефолтов по категории 2

#доли дефолтов по каждой категории
train_df['defaults_by_home_ownership_cat'] = 0
train_df.loc[train_df['home_ownership_cat'] == 0, 'defaults_by_home_ownership_cat'] = total_defaults / home_ownership_cat_0
train_df.loc[train_df['home_ownership_cat'] == 1, 'defaults_by_home_ownership_cat'] = total_defaults / home_ownership_cat_1
train_df.loc[train_df['home_ownership_cat'] == 2, 'defaults_by_home_ownership_cat'] = total_defaults / home_ownership_cat_2

In [10]:
#4. Новые признаки на основе взаимодействия признаков:
#используя деление, разность, сумму, произведение разных числовых признаков
train_df['credit_history_credit_score'] = train_df['Years of Credit History'] * train_df['Credit Score']
#категория заменяется на статистику другого признака в этой категории (min, max, mean, std)
annual_income_median_cat_0 = train_df.loc[train_df['home_ownership_cat'] == 0, 'Annual Income'].median() 
annual_income_median_cat_1 = train_df.loc[train_df['home_ownership_cat'] == 1, 'Annual Income'].median() 
annual_income_median_cat_2 = train_df.loc[train_df['home_ownership_cat'] == 2, 'Annual Income'].median() 
train_df.loc[train_df['home_ownership_cat'] == 0, 'home_ownership_annual_income_median'] = annual_income_median_cat_0
train_df.loc[train_df['home_ownership_cat'] == 1, 'home_ownership_annual_income_median'] = annual_income_median_cat_1
train_df.loc[train_df['home_ownership_cat'] == 2, 'home_ownership_annual_income_median'] = annual_income_median_cat_2

#### c) Отобрать лучшие признаки (feature selection)

In [11]:
X = train_df.drop(columns=target)
y = train_df[target]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, shuffle=True, random_state=22)
catb = catboost.CatBoostClassifier(silent=True, random_state=22)
features = X.columns

In [12]:
#выбрал RFE так как отбор автоматизирован
rfe = RFE(estimator=catb, n_features_to_select=10)
X_selected_RFE = rfe.fit_transform(X, y) 
selected_features_RFE = X.columns[rfe.support_] 

In [14]:
is_chosen = np.where(rfe.support_, '✅', '❌')
results = pd.DataFrame({'feature': features, 'rank': rfe.ranking_, 'is_chosen': is_chosen})
results = results.sort_values('rank', ascending=True)
results

,feature,rank,is_chosen
20,defaults_by_home_ownership_cat,1,✅
18,had_bankruptcies,1,✅
3,Number of Open Accounts,1,✅
4,Years of Credit History,1,✅
5,Maximum Open Credit,1,✅
17,budget_burdening,1,✅
7,Months since last delinquent,1,✅
16,social_class,1,✅
9,Term,1,✅
15,loan_part,1,✅


Вывод. Из 8-ми созданных мной признаков 5 были отобраны как лучшие (rank=1).